In [11]:
import os
import sys
import clr
import time
import pandas as pd
import numpy as np
from tqdm import tqdm

clr.FindAssembly('PyConnDH')
clr.AddReference('PyConnDH')
from PyConnDH import *

class mmm():
    def __init__(self,):
        self.path='E:\\Anaconda3\\PyComDH\\'
        self.ppa=pd.read_csv(self.path+'1.Pipe.csv',header=0,encoding='gbk')  #读取原始管道数据
        
    def setpipe(self):
        pp=self.ppa
        ppdia=pd.concat([pp.loc[range(0,68),['内径']]*0.01*i for i in range(5,96)],axis=0).reset_index()  #设置管径
        ppdia=ppdia['内径']
        ppcoe=pd.concat([pd.DataFrame(np.ones([68,1]))*0.01*i for i in range(5,96)],axis=0).reset_index()#生成管径系数
        ppcoe=ppcoe[0]
        ppcode=(pd.concat([pp.loc[range(0,68),['编号']] for i in range(5,96)],axis=0)).reset_index()#生成管道编号
        ppcode=ppcode['编号']
        pplabel=(pd.concat([pp.loc[range(0,68),['编号']] for i in range(5,96)],axis=0)-1).reset_index()#生成管道标签
        pplabel=pplabel['编号']

        ppc=pd.concat([ppcode,pplabel,ppdia,ppcoe],axis=1)
        ppc.columns=['Code','label','InnerDiameter','DN']
        return ppc
    def calc(self,i):
        csvOutPath = sys.executable.split("python.exe")[0] + "PyComDH\\out"
        csvInPath = sys.executable.split("python.exe")[0] + "PyComDH\\in"
        if not os.path.exists(self.path+'out\\'+str(i)):
            os.mkdir(self.path+'out\\'+str(i))
            
        calcInfo = calcDH.doCal(str(i))
        if calcInfo.split(",")[0] == 'false':  # 计算异常
            print('第'+str(i)+'次计算失败！'+ calcInfo.split(",")[1])  # 控制台输出异常问题
        else:
            print('第'+str(i)+'次计算：成功')
            
    def run(self):
        ppcal=self.setpipe()
        startT=time.time()
        for i in range(2402,2403):
            ppb=self.ppa.copy()#初始化管径数据
            ppb.loc[ppcal.iloc[i,1],['内径']]=ppcal.iloc[i,2] #修改管径
            ppb.to_csv(self.path+'in\\1.Pipe.csv',index=None,encoding='gbk')
            
            self.calc(i)
            
            stopT=time.time()
            print('总共用时s：',round(stopT-startT,4))

In [12]:
M=mmm()
M.run()

第2284次计算：成功
总共用时s： 1.0347


In [10]:
# 计算模块
def calc():
    csvOutPath = sys.executable.split("python.exe")[0] + "PyComDH\\out"  # 获取csv所在文件夹路径的方法,DLL里是自动获取了Application的路径，所以py调用时就是在python.exe的那个目录
    csvInPath = sys.executable.split("python.exe")[0] + "PyComDH\\in"

    calcInfo = calcDH.doCal('0')  # 开始计算
    if calcInfo.split(",")[0] == 'false':  # 计算异常
        print("调用计算失败：" + calcInfo.split(",")[1])  # 控制台输出异常问题
    else:
        print("调用计算成功")


if __name__ == '__main__':
    calc()

调用计算成功
